In [75]:
import datetime as dt
import json
import re
import time

import datazimmer as dz
import pandas as pd

from aswan import get_soup, get_json, add_url_params
from tqdm import tqdm

In [4]:
fifa_rank_base = dz.SourceUrl("https://www.fifa.com/fifa-world-ranking/men")

2023-01-30 11:04.13 [warning  ] can't get module name, likely due to notebook call


In [5]:
soup = get_soup(fifa_rank_base)

In [6]:
dr = '"dates"\:\[.*?\]'

In [12]:
date_ids = json.loads("{" + re.findall(dr, str(soup))[0] + "}")

In [103]:
ranks_by_date = {}
for date_dic in tqdm(date_ids["dates"]):
    did = date_dic["id"]
    ranks_by_date[did] = get_json(
        "https://www.fifa.com/api/ranking-overview",
        params={"locale": "en", "dateId": date_dic["id"]},
    )
    time.sleep(0.1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 323/323 [03:34<00:00,  1.51it/s]


In [55]:
uefa_seasons = list(range(2005, dt.date.today().year + 1))

In [71]:
outer_keys = ["countryName", "displayName"]

all_ranks = []
for season_year in tqdm(uefa_seasons):
    for pref in [
        "coefficientType=MEN_ASSOCIATION&coefficientRange=SEASONAL",
        "coefficientType=MEN_CLUB&coefficientRange=OVERALL",
    ]:
        url = (
            f"https://comp.uefa.com/v2/coefficients?{pref}&"
            f"seasonYear={season_year}&page=1&pagesize=500&language=en"
        )

        club_json = get_json(url)

        for member in club_json["data"]["members"]:
            outer_d = {k: member["member"][k] for k in outer_keys}
            for ranks in member["seasonRankings"]:
                all_ranks.append(outer_d | ranks)
    time.sleep(1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:28<00:00,  1.49s/it]


In [76]:
pd.DataFrame(all_ranks).to_csv(dz.get_raw_data_path("uefa-ranks.csv"), index=None)

In [104]:
date_dic = {d["id"]: d["text"] for d in date_ids["dates"]}

In [105]:
todrop = ["flag", "tag"]

In [106]:
def flat(d0):
    d = d0.copy()
    ri = d.pop("rankingItem")
    return {k: v for k, v in (ri | d).items() if k not in todrop}

In [107]:
fifa_dfs = []
for k, v in ranks_by_date.items():
    fifa_dfs.append(pd.DataFrame(map(flat, v["rankings"])).assign(date=date_dic[k]))

In [110]:
pd.concat(fifa_dfs, ignore_index=True).to_csv(
    dz.get_raw_data_path("fifa-ranks.csv"), index=None
)